In [ ]:
import pandas as pd
import copy
from matplotlib import pyplot as plt

In [ ]:
params = {'k':0.8, 'l':2.0, 'kg':0.1, 'lg':0.2, 'pw1':1.4, 'pw2':1.6, 'f':1.0}

taxes = {'ty':0.0, 'tc':0.1, 'tm':0.0}  #ty: '0' is an initial value (endogenous)

sys_params = {'x1':0, 'x2':0, 'k1':0, 'k2':0, 'l1':0, 'l2':0, 'p1':0, 'q1':0, 'p2':0, 'r':0, 'w':0, 'uk1':0, 'uk2':0, 'ul1':0, 'ul2':0, 'y':0, 'c1':0, 'c2':0, 'm':0, 'e':0, 'bp':0, 'g':0, 't':0, 'PSB':0}

result_df = pd.DataFrame([], columns=sys_params.keys())
dict_list = []

def system(params, taxess):
    global sys_params

    g2 = (params['kg']**0.5) * (params['lg']**0.5)
    kx = params['k'] - params['kg']
    lx = params['l'] - params['lg']
    
    sys_params['p1'] = params['pw1'] * params['f']
    sys_params['q1'] = sys_params['p1'] + taxes['tc']
    sys_params['p2'] = params['pw2'] * params['f'] + taxes['tm']

    sys_params['r'] = ((sys_params['p2']**3) * ((1/3)**0.75 + 3**0.25)**2) / (8 * sys_params['p1']**2)
    sys_params['w'] = ( 2 * (sys_params['p1']**2)) / (sys_params['p2'] * ((1/3)**0.75 + 3**0.25)**2)

    sys_params['ul1'] = ( 3 * sys_params['r'] / sys_params['w'])**0.25
    sys_params['ul2'] = (sys_params['r'] / sys_params['w']**2)**0.5
    sys_params['uk1'] = (sys_params['w'] / (3 * sys_params['r']))**0.75
    sys_params['uk2'] = (sys_params['w'] / sys_params['r'])**0.5

    a = sys_params['ul1'] * sys_params['uk2'] - sys_params['ul2'] * sys_params['uk1']
    sys_params['x1'] = (lx * sys_params['uk2'] - kx * sys_params['ul2']) / a
    sys_params['x2'] = (kx * sys_params['ul1'] - lx * sys_params['uk1']) / a

    sys_params['k1'] = sys_params['uk1'] * sys_params['x1']
    sys_params['k2'] = sys_params['uk2'] * sys_params['x2']

    sys_params['l1'] = sys_params['ul1'] * sys_params['x1']
    sys_params['l2'] = sys_params['ul2'] * sys_params['x2']

    sys_params['y'] = sys_params['r'] * params['k'] + sys_params['w'] * params['l']  - taxes['ty']

    sys_params['c1'] = sys_params['y'] / ( 2 * sys_params['q1'] )
    sys_params['c2'] = sys_params['y'] / ( 2 * sys_params['p2'] )
    
    sys_params['m'] = sys_params['c2'] - sys_params['x2'] - g2
    sys_params['e'] = sys_params['x1'] - sys_params['c1']
    sys_params['bp'] = params['pw1'] * sys_params['e'] - params['pw2'] * sys_params['m'] # Export: Good1, Import: Good2

    sys_params['g'] = sys_params['p2'] * g2 - sys_params['r'] * params['kg'] - sys_params['w'] * params['lg']
    sys_params['t'] = taxes['ty'] + taxes['tc'] * sys_params['c1'] + taxes['tm'] * sys_params['m']
    
    sys_params['PSB'] = sys_params['g'] + sys_params['t']
    

In [ ]:
system(params, taxes)

N = 0

In [ ]:
while abs(sys_params['PSB']) >= 0.001:
    
    N += 1
    print(N)
    print("Iterating!")
    print(params)
    print(taxes)
    print(sys_params)
    dict_list.append(copy.copy(sys_params))
    #global taxes
 
    if sys_params['PSB'] > 0:
        taxes['ty'] = taxes['ty'] - 0.0001
        system(params, taxes)
        continue
        
    elif sys_params['PSB'] < 0:
        taxes['ty'] = taxes['ty'] + 0.0001
        system(params, taxes)
        continue
        
    else:
        pass
                       
else:
    print("Iterated!")
    print(taxes)
    print(params)
    print(sys_params)
    dict_list.append(copy.copy(sys_params))
    print([sys_params['g'], sys_params['t']])


In [ ]:
result_df = pd.concat([result_df, pd.DataFrame.from_dict(dict_list)])
print(result_df[['g', 't']])

result_df[['g', 't']].plot()
plt.show()
